## FCN -  `detect()` Process 

We generate the inputs to `DetectTargetLayer` , to manipulate and modify the layer to procduce a modified `output_rois` 
containing false positives. 
This will be passed on the the heatmap layer, and through there will become the input to FCN.

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys,os, pprint
pp = pprint.PrettyPrinter(indent=2, width=100)
print('Current working dir: ', os.getcwd())
if '..' not in sys.path:
    print("appending '..' to sys.path")
    sys.path.append('..')
    
import numpy as np
import mrcnn.utils     as utils
import mrcnn.visualize as visualize
from mrcnn.prep_notebook import build_fcn_evaluate_pipeline, get_evaluate_batch
# from mrcnn.prep_notebook import get_inference_batch, get_image_batch, get_training_batch
from mrcnn.visualize     import display_training_batch
from mrcnn.coco          import prep_coco_dataset
from mrcnn.datagen import data_gen_simulate

input_parms  =" --batch_size     1  "
input_parms +=" --mrcnn_logs_dir train_mrcnn_coco "
input_parms +=" --fcn_logs_dir   train_fcn32_adam " 
input_parms +=" --mrcnn_model    last "
input_parms +=" --fcn_model      last "
input_parms +=" --fcn_arch       fcn8 " 
input_parms +=" --coco_classes   78 79 80 81 82 44 46 47 48 49 50 51 34 35 36 37 38 39 40 41 42 43 10 11 13 14 15 "
input_parms +=" --sysout         screen "

# input_parms = " --epochs         2   --steps_in_epoch 32  --last_epoch 0 "
# input_parms +=" --lr 0.00001     --val_steps 8 " 
# input_parms +=" --opt            adam "
# input_parms +=" --fcn_layers     all " 
# input_parms +=" --new_log_folder    "
parser = utils.command_line_parser()
args = parser.parse_args(input_parms.split())


Current working dir:  E:\git_projs\MRCNN3\notebooks
appending '..' to sys.path


In [ ]:
# del mrcnn_model, fcn_model
mrcnn_model, fcn_model = build_fcn_evaluate_pipeline(args = args,verbose = 0)


In [ ]:
# mrcnn_model.config.display()
fcn_model.keras_model.summary()

### Load dataset

In [ ]:
dataset_test, test_generator = prep_coco_dataset(["train", 'val35k'], mrcnn_model.config, generator = True, shuffle = False, return_coco =True, load_coco_classes=args.coco_classes)
class_names = dataset_test.class_names 
####  Display number of loaded images and class names
print("Testing Dataset Image Count: {}".format(len(dataset_test.image_ids)))
print("Testing Dataset Class Count: {}".format(dataset_test.num_classes))

## Display Images

### Get next image from generator and display 

In [ ]:
test_batch_x, test_batch_y = next(test_generator)
display_training_batch(dataset_test, test_batch_x)

### info about some images

In [ ]:
## 62642 (persons),   68539 (trucks) 36466 (surfers)  75040 (boat and persons)
## 36466 surfers. 5498 basketbal players, 27711,30531
## 5498 lots of motorcylces & persons - 
## Persons: #26026, #7719, 111864, 58240,  
## 35347 - laptops, keyboards and cat
## items = [59199 , 102868]
## 89243: Person, bicylce and traiffic lights
## 101623 (cake and forks), 41423 (elephant & people)
## 33477 Table, bowl, cup, sandwich, knife
## 26763 Table, chair, cup , bowl, bottle
## 49316 Table, book, backpack, many bottles
## 12902 Car , truck and traffic light tree
## 57327 person and horses
## 26026 -- mountain, skiers, skiboards 
##  7719 -- people, bottles, handbags
## 27711 -- persons and boats
## 18045 -- person     
## 36769 -- people in meeting room with a 'giraffe'detection?
##  8634 -- person
## 30531 -- tennis players 
##  8645 -- two people
## 24627 -- Pizza, bowl, cups
# IMAGE_LIST = [75040] 
# IMAGE_LIST = [89243]
# IMAGE_LIST = [33477]
# IMAGE_LIST = [29731]
# IMAGE_LIST = [26763]
# IMAGE_LIST = [89243]  ## 89243: Person, bicylce and traiffic lights
# IMAGE_LIST = [43168]   ## 43168 Multiple umbrella, chairs, beds

# IMAGE_LIST = [18388, 64305] # , 1507]
# IMAGE_LIST = [107639, 112710, 39775]
# IMAGE_LIST = [24627, 33758, 6749]
# IMAGE_LIST = [24627, 33758] # , 6749]

### Get images using a list of image ids

In [ ]:
# from mrcnn.prep_notebook import get_training_batch, get_inference_batch, get_evaluate_batch
IMAGE_LIST = [2]
eval_batch = get_evaluate_batch(dataset_test, mrcnn_model.config, generator = test_generator, display = True)
# eval_batch = get_evaluate_batch(dataset_test, mrcnn_model.config, IMAGE_LIST, display = True)


# image_batch = get_image_batch(dataset_test,IMAGE_LIST, display=True)
# inference_batch, image_batch = get_inference_batch(dataset_test, mrcnn_model, IMAGE_LIST, display = True)

In [ ]:
print(type(eval_batch), len(eval_batch))
for item in eval_batch :    
    if isinstance(item, list):
        print(' list ')
        for i in item:
            print('     ',i.shape) 
    else:
        print(type(item), item.shape)
print(eval_batch[2])        

### Other display functions

#### Display top masks for sample images

In [ ]:
# Load and display random samples
image_ids = np.random.choice(dataset_test.image_ids, 7)
for image_id in image_ids:
    image = dataset_test.load_image(image_id)
    mask, class_ids = dataset_test.load_mask(image_id)
    visualize.display_top_masks(image, mask, class_ids, dataset_test.class_names)

####  Dispaly a random image from dataset_test

In [ ]:
# Load random image and mask. ### 27711 persons and boats
image_id = np.random.choice(dataset_test.image_ids)
_ = batch_input = demo_image(dataset_test, mrcnn_model.config, image_id, masks = False)

#### Get an image and place into train_batch_x

In [ ]:
IMAGE_LIST= [30531]
batch_input = get_input_batch(dataset_test, mrcnn_model.config, 30531, masks = False)

## MRCNN detection using `mrcnn.detect()` 

In [ ]:
# fcn_results = fcn_model.detect(mrcnn_model, image_batch)
mrcnn_results = mrcnn_model.evaluate(eval_batch, verbose =1)

In [ ]:
float_formatter = lambda x: "%9.4f" % x
np_format = {}
np_format['float']=float_formatter
np.set_printoptions(linewidth=195, precision=3, floatmode='fixed', threshold =10000, formatter = np_format)

print(len(mrcnn_results))
r = mrcnn_results[0]
print(r.keys())
for i in sorted(r.keys()):
     print('{:20s}    {}'.format(i , r[i].shape))
print('-----------------------------')        
print('  image_meta     : ', r["image_meta"].shape, r["image_meta"][:11] )
print('  rois           : ', r['rois'].shape)
# print(r['rois'])
print('  class ids      : ', r['class_ids'].shape, r['class_ids'])
print('  scores         : ', r['scores'].shape)
print('  pr_heatmap     : ', r['pr_hm'].shape) 
print('  detection_ind  : ', r['detection_ind'].shape, r['detection_ind'])
print('  pr_scores      : ', r['pr_scores'].shape)
# for i in r['pr_scores']:
#     print(i[:12])
#     print('\t\t\t', i[12:18] , i[18:])
print(r['detections'])

## FCN detection using `fcn.detect()` 

In [ ]:
# fcn_results = fcn_model.detect(mrcnn_model, image_batch)
fcn_results = fcn_model.evaluate(mrcnn_model, eval_batch, verbose =1)

In [ ]:
np.set_printoptions(linewidth=180,precision=4,threshold=10000, suppress = True)
print(len(fcn_results))
f = fcn_results[0]
print(f.keys())
for i in sorted(f.keys()):
     print('{:20s}    {}'.format(i , f[i].shape))
# for i in r.keys():
#     print(i , r[i].shape)
# print('  image          : ', f["image"].shape )
# print('  image_meta     : ', f["image_meta"].shape, f["image_meta"][:11] )
# print('  rois           : ', f['rois'].shape)
# print(f['rois'])
# print('  class ids      : ', f['class_ids'].shape, f['class_ids'])
# print('  scores         : ', f['mrcnn_scores'].shape)
# print('  pr_heatmap     : ', f['pr_hm'].shape)
print('  fcn_hm         : ', f['fcn_hm'].shape)
print('  fcn_sm         : ', f['fcn_sm'].shape)
# print('  pr_scores      : ', f['pr_scores'].shape)
print('  pr_scores  : ')
print(f['pr_scores'][:,:10])
print('  fcn_scores:',f['fcn_scores'].shape)
# print('  fcn_scores  : ')
print(f['fcn_scores'])
# 
# print('  pr_scores_by_class:',r['pr_scores_by_class'].shape)
# print(r['pr_scores_by_class'][ r['class_ids'],:10])
# print('  fcn_scores_by_class:',r['fcn_scores_by_class'].shape)
# print(r['fcn_scores_by_class'][ r['class_ids'],:10])
# print(f['detections'])


### Display FCN detections

In [ ]:
np_format = {}
float_formatter = lambda x: "%10.4f" % x
int_formatter   = lambda x: "%10d" % x
np_format['float'] = float_formatter
np_format['int']   = int_formatter
np.set_printoptions(linewidth=195, precision=4, floatmode='fixed', threshold =10000, formatter = np_format)

In [ ]:
# float_formatter = lambda x: "%10.4f" % x
# np_format = {}
# np_format['float']=float_formatter
# np.set_printoptions(linewidth=195, precision=3, floatmode='fixed', threshold =10000, formatter = np_format)
# for i in range(results[0]['pr_scores'].shape[0]):
#     print(results[0]['pr_scores'][i,7:])

In [ ]:
for i, [molded_bbox, cls, scr, pr_scr, fcn_scr] in enumerate(zip(f['molded_rois'].astype(np.int), f['class_ids'],  f['scores'], f['pr_scores'], f['fcn_scores'])):
    
    print('{} {} {:2d}  {:.<18s}  {:5.4f} {}  '.format(i, molded_bbox, cls, dataset_test.class_names[cls], scr, fcn_scr[[4,5,6,7,8]]))
    print('{:>86s} {}'.format(' mrcnn old style scores:  ',  pr_scr[[9,10,11]]))
    print('{:>86s} {}'.format('   fcn old style scores:  ', fcn_scr[[9,10,11]]))
    print()
    print('{:>86s} {}'.format('      mrcnn alt scores1:  ', pr_scr[[12,13,14,15,16,17]]))
    print('{:>86s} {}'.format('        fcn alt scores1:  ', fcn_scr[[12,13,14,15,16,17]]))
    print()
    print('{:>86s} {}'.format('          mrcnn_scores2:  ', pr_scr[[18,19,20,21,22,23]]))
    print('{:>86s} {}'.format('            fcn_scores2:  ', fcn_scr[[18,19,20,21,22,23]]))
    print()

#### detections as returned from the model's `detect()` functon

In [ ]:
lmt =15
f = fcn_results[0]
print('       classes :', f['class_ids'][:lmt])
names  = " ".join([ '{:>10s}'.format(class_names[i][-10:]) for i in f['class_ids'][:lmt]])

print('               : ', names)
print('                ', f['detection_ind'][:lmt])
print('   orig scores :',f['scores'][:lmt])
print('   norm scores :', f['pr_scores'][:lmt,8])
# print('  pr_scores[5] :', f['pr_scores'][:,5])

print('-'*185)
print('    pr_scrs[8] :', f['pr_scores'][:lmt,11])
print(' fcn_scores[8] :', f['fcn_scores'][:lmt,11])
print()
print(' pr_scores[13] :', f['pr_scores'][:lmt,14])
print('fcn_scores[13] :', f['fcn_scores'][:lmt,14])
print()
print(' pr_scores[19] :', f['pr_scores'][:lmt,20])
print('fcn_scores[19] :', f['fcn_scores'][:lmt,20])

In [ ]:
# r = results[0]
print(f['detections'].shape)
print('   old_scores: (gauss. sum over large bbox/bbox area/ gauss_sum * normlzd_score))')
print('  alt_scores 1: (gauss. sum over small mask/mask area/ gauss_sum / mask_area):  ')
sort_by_class_order = np.argsort(f['class_ids'])


# for i in range(len( f['class_ids'])):
for i in sort_by_class_order:
#     print(i , f['rois'][i].astype(np.float), f['scores'][i], f['class_ids'][i], class_names[f['class_ids'][i]])
#     print(i , f['detections'][i], f['class_ids'][i], class_names[f['class_ids'][i]])
    det_type = '       --> ADDED FP ' if f['pr_scores'][i,6] == -1 else '      Original detection'
    print(i , f['pr_scores'][i,:9], f['pr_scores'][i,4], class_names[ f['pr_scores'][i,4].astype(np.int)], det_type) 
    print()
    print(i , 'pr: old_scores [9,10,11]:  '.rjust(90), f['pr_scores'][i,9:12])    
    print(i , '  alt_scores 1 [12 - 17]:  '.rjust(90), f['pr_scores'][i,12:18])    
    print(i , '  alt_scores 2 [18 - 23]:  '.rjust(90), f['pr_scores'][i,18:23])    
    print(i)
#     print(i , f['fcn_scores'][i,:8], f['fcn_scores'][i,4], class_names[ f['fcn_scores'][i,4].astype(np.int)])    
    print(i , 'fcn: old_scores [9,10,11]:  '.rjust(90), f['fcn_scores'][i,9:12])    
    print(i ,  '  alt_scores 1 [12 - 17]:  '.rjust(90), f['fcn_scores'][i,12:18])    
    print(i ,  '  alt_scores 2 [18 - 23]:  '.rjust(90), f['fcn_scores'][i,18:23])    
    print()
    print(i , '  old score [11]:  '.rjust(90), ' from mrcnn:{:10.4f}  from FCN: {:10.4f} '.format(f['pr_scores'][i,11] ,f['fcn_scores'][i,11]))
    print(i , 'alt score 1 [14]:  '.rjust(90), ' from mrcnn:{:10.4f}  from FCN: {:10.4f} '.format(f['pr_scores'][i,14],f['fcn_scores'][i,14]))
    print(i , 'alt score 2 [20]:  '.rjust(90), ' from mrcnn:{:10.4f}  from FCN: {:10.4f} '.format(f['pr_scores'][i,20],f['fcn_scores'][i,20]))
    print()

In [ ]:
f = fcn_results[0]
print('Image Meta: ', f['image_meta'][:10])
visualize.display_instances(f['image'], f['rois'],  f['class_ids'], dataset_test.class_names, f['scores'], only_classes=[27],
                             title="MRCNN Predictions", score_range=(0.0, 1.7))


In [ ]:
#### Display detections
## old_score   : 11
## alt_score_1 : 14
## alt_score_2 : 20
from mrcnn.utils import log # Display results
# ax = visualize.get_ax(rows =1, cols = 1, size= 20)
r = fcn_results[0]
clses = [27]
class_ids = np.unique(r['pr_scores'][:,4]).astype(int).tolist()
print('class ids: ', class_ids)
print('Image Meta: ', r['image_meta'][:10])

# visualize.display_instances(r['image'], r['rois'],  r['class_ids'], dataset_test.class_names, r['scores'], title="Predictions", score_range=(0.7, 0.99), size = 24)
# visualize.display_instances(r['image'], r['rois'],  r['class_ids'], class_names, r['scores'], title="Predictions", score_range=(0.0, 0.99), only_classes= [42], size = 24)
visualize.display_instances_two_scores(r['image'], r['rois'], r['class_ids'], class_names, r['pr_scores'][:,11], r['fcn_scores'][:,11],title="Predictions", only_classes=clses,size = 28)
visualize.display_instances_two_scores(r['image'], r['rois'], r['class_ids'], class_names, r['pr_scores'][:,14], r['fcn_scores'][:,14],title="Predictions", only_classes=clses,size = 28)
visualize.display_instances_two_scores(r['image'], r['rois'], r['class_ids'], class_names, r['pr_scores'][:,20], r['fcn_scores'][:,20],title="Predictions", only_classes=clses,size = 28)

# Display Heatmaps

In [ ]:
from mrcnn.visualize import (plot_one_bbox_heatmap, 
                             plot_3d_heatmap, plot_2d_heatmap, 
                              plot_2d_heatmap_compare, plot_3d_heatmap_compare)
# import matplotlib as plt
# %matplotlib inline
img_id = 0
image_id=f['image_meta'][0]
print('Image id: ',image_id, ' Coco ID: ', dataset_test.image_info[image_id]['id'])
print(f['pr_scores'][:,4])
coco_class_names = dataset_test.class_names
pr_class_ids = np.unique(f['pr_scores'][:,4]).astype(int).tolist()
fcn_class_ids = np.unique(f['fcn_scores'][:,4]).astype(int).tolist()
print('Pred class ids: ', pr_class_ids)
print('FCN  class ids: ', fcn_class_ids)

## 2D plots

In [ ]:
# visualize.plot_2d_gaussian(np.sum(r['pr_hm'], axis=-1)
fig = visualize.plot_2d_heatmap(r['pr_hm'], r['pr_hm_scores'], 0, class_ids = pr_class_ids,
                                class_names = class_names, columns = 4, scale = 4, scaling = 'none')

###   FCN heatmaps

In [ ]:
fig = visualize.plot_2d_heatmap(f['fcn_hm'], f['fcn_scores_by_class'], 0,
                                class_names = class_names, columns = 4, scale = 4, scaling = 'class')

## Display Images overlaid with heatmaps

### Overlay image with pred_heatmaps

In [ ]:
from mrcnn.visualize import display_heatmaps, display_heatmaps_fcn, display_heatmaps_mrcnn, inference_heatmaps_display
# visualize.display_image_bw(image)
# print(model_gt_heatmap_scores.shape)
class_ids = np.unique(f['pr_scores'][:,4]).astype(int).tolist()
print('class ids: ', class_ids)
inference_heatmaps_display(fcn_results, 0, class_ids = class_ids, class_names = coco_class_names, hm = 'pr_hm' ,config = mrcnn_model.config, scaling = 'class') 
# , display_heatmaps_mrcnn(train_batch_x, model_output, 0, hm = 'pr',  
#                      config = mrcnn_config, class_ids = class_ids)

### Overlay image with fcn_heatmaps

In [ ]:
from mrcnn.visualize import display_heatmaps, display_heatmaps_fcn, display_heatmaps_mrcnn, inference_heatmaps_display
# visualize.display_image_bw(image)
# print(model_gt_heatmap_scores.shape)
class_ids = np.unique(f['pr_scores'][:,4]).astype(int).tolist()
print('class ids: ', class_ids)
inference_heatmaps_display(fcn_results, 0, class_ids = class_ids, class_names = coco_class_names, hm = 'fcn_hm' ,config = mrcnn_model.config, scaling = 'class') 
# , display_heatmaps_mrcnn(train_batch_x, model_output, 0, hm = 'pr',  

### Overlay image with pr_heatmaps and fcn_heatmaps

In [ ]:
visualize.inference_heatmaps_compare(fcn_results, image_id = 0 , hm = 'fcn_hm', 
                     config = mrcnn_model.config, class_ids = pr_class_ids, class_names = coco_class_names, scaling = 'each')

##  3D Plots

####  3D plot of `fcn_heatmap` returned form model - classes predicted by MRCNN only 

In [ ]:
print(np.max(f['fcn_hm']), np.min(f['fcn_hm']))

print(f['fcn_hm'].shape)
class_ids = np.unique(f['pr_scores'][:,4]).astype(int).tolist()
print('Image : {}  ClassIds: {}'.format(img_id, class_ids))
# plot_3d_heatmap(f['fcn_hm'], 0, class_ids,class_names = coco_class_names, scaling = 'none')

####  3D plot of `fcn_softmax` returned form model - only classes in pred_scores

In [ ]:
for img_id in [0]: ##range(mrcnn_config.BATCH_SIZE):
    print(f['fcn_hm'].shape)
    class_ids = np.unique(f['pr_scores'][:,4]).astype(int).tolist()
    print('Image : {}  ClassIds: {}'.format(img_id, class_ids))
    plot_3d_heatmap(f['fcn_sm'], 0, class_ids,class_names = coco_class_names, scaling = 'none', columns = 1)

## Display results from `detect()`

### Compare mrcnn predcited and FCN generated heatmaps from inference process

In [ ]:
class_ids = np.arange(20)
visualize.inference_heatmaps_compare(fcn_results, img_idx, class_ids = None, class_names=coco_class_names, hm = 'hm', config = mrcnn_model.config,scaling = 'each')

In [ ]:
img_id = 0
r = fcn_results[img_id]
pr_class_ids = np.unique(r['class_ids'])
fig = visualize.plot_2d_heatmap(r['fcn_hm'], r['fcn_scores_by_class'], img_id, pr_class_ids, columns = 3, num_bboxes = 0,
                          class_names = coco_class_names, scale = 4, scaling = 'none')

####  3D comparative display `pred_heatmap` / `fcn_heatmap` 

In [ ]:
img_id = 0
r = fcn_results[img_id]
print(r['pr_scores_by_class'].ndim)
# print('Image id: ',image_id, ' Coco ID: ', dataset_train.image_info[image_id]['id'])
pr_class_ids = np.unique(r['pr_scores_by_class'][:,:,4]).astype(int).tolist()
print('Image : {}  Predicted    ClassIds: {}'.format(img_id, pr_class_ids))

visualize.plot_3d_heatmap_compare(r['pr_hm'], r['fcn_hm'], 0, class_ids = pr_class_ids, 
                            class_names = coco_class_names, size=(8,8), zlim = 'all' , scaling = 'none')    